In [1]:
from sklearn.model_selection import ShuffleSplit

from mne import Epochs, pick_types, events_from_annotations
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci


import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Subset
from torch import nn
import torch.nn.functional as F
from torch.utils.data import RandomSampler
from torch.utils.data import SequentialSampler


import os, sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), os.pardir, "src"))) 
from common import EpochsDataset


from common import train

c:\users\giulia pezzutti\appdata\local\programs\python\python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import pandas as pd
import numpy as np
import pickle


# #############################################################################
# # Set parameters and read data

# avoid classification of evoked responses by using epochs that start 1s after
# cue onset.


def get_data():
    tmin, tmax = -1., 4.
    event_id = dict(hands=2, feet=3)
    subject = 1
    runs = [6, 10, 14]  # motor imagery: hands vs feet

    raw_fnames = eegbci.load_data(subject, runs)
    raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])

    # strip channel names of "." characters
    raw.rename_channels(lambda x: x.strip('.'))

    # Apply band-pass filter
    raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')

    events, _ = events_from_annotations(raw)

    picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                       exclude='bads')

    # Read epochs (train will be done only between 1 and 2s)
    # Testing will be done with a running classifier
    epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                    baseline=None, preload=True)
    epochs.crop(tmin=1., tmax=None)
    labels = epochs.events[:, 2] - 2
    return epochs.get_data()[:, :, :256], labels


def get_paradigm_data():

    path_ratings = '../data/ratings-results/ratings-results.csv'
    df_ratings = pd.read_csv(path_ratings, index_col=0)

    code = 'vamo24'

    with open('../data/pickle/' + code + '_data.pkl', 'rb') as f:
        data = pickle.load(f)
    with open('../data/pickle/' + code + '_info.pkl', 'rb') as f:
        info = pickle.load(f)
        index_channels_eeg = [index for index in range(len(info['channels'])) if
                              'EOG' not in info['channels'][index]]
        channels_eeg = np.array(info['channels'])[index_channels_eeg]
    with open('../data/pickle/' + code + '_labels.pkl', 'rb') as f:
        labels = pickle.load(f)
        conditions = [label.split('/')[1] for label in labels]
        unique_conditions = list(set(conditions))

    for idx, epoch in enumerate(data):
        data[idx] = (epoch - np.mean(epoch)) / np.std(epoch)

    data_ratings = df_ratings.loc[df_ratings['code'] == code]
    ratings = []
    label_binary = []

    for idx, _ in enumerate(data):
        img_name = labels[idx].split('/')[0]
        ratings.append(data_ratings.loc[data_ratings['img_name'] == img_name][['valence', 'arousal']].values[0])

    for label in ratings:
        valence = label[0]

        if valence > 0:
            label_binary.append(1)
        else:
            label_binary.append(0)

    return data, np.array(label_binary)

epochs_data, labels = get_paradigm_data()
print(epochs_data.shape)
print(labels.shape)

(595, 32, 751)
(595,)


In [9]:
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data)
train_idx, test_idx = next(cv_split)

In [10]:
def scale(X):
    """Standard scaling of data along the last dimention.

    Parameters
    ----------
    X : array, shape (n_channels, n_times)
        The input signals.

    Returns
    -------
    X_t : array, shape (n_channels, n_times)
        The scaled signals.
    """
    return X / 2e-5

In [11]:
dataset = EpochsDataset(epochs_data, labels, transform=scale)

ds_train, ds_valid = Subset(dataset, train_idx), Subset(dataset, test_idx)

batch_size_train = len(ds_train)
batch_size_valid = len(ds_valid)
sampler_train = RandomSampler(ds_train)
sampler_valid = SequentialSampler(ds_valid)

# create loaders
num_workers = 0
loader_train = \
    DataLoader(ds_train, batch_size=batch_size_train,
               num_workers=num_workers, sampler=sampler_train)
loader_valid = \
    DataLoader(ds_valid, batch_size=batch_size_valid,
               num_workers=num_workers, sampler=sampler_valid)


In [12]:
dataset = EpochsDataset(epochs_data, labels, transform=scale)

ds_train, ds_valid = Subset(dataset, train_idx), Subset(dataset, test_idx)

batch_size_train = len(ds_train)
batch_size_valid = len(ds_valid)
sampler_train = RandomSampler(ds_train)
sampler_valid = SequentialSampler(ds_valid)

# create loaders
num_workers = 0
loader_train = \
    DataLoader(ds_train, batch_size=batch_size_train,
               num_workers=num_workers, sampler=sampler_train)
loader_valid = \
    DataLoader(ds_valid, batch_size=batch_size_valid,
               num_workers=num_workers, sampler=sampler_valid)

Model definition

In [13]:
class CommonSpatialFilterModel(nn.Module):
    """The model implements a CSP-like network for BCI applications

    Parameters
    ----------
    spatial_dim : int
        Number of channels
    n_components : int
        The number of spatial filters.
    """
    def __init__(self, spatial_dim, n_components=5):
        super().__init__()
        self.spatial_dim = spatial_dim
        self.n_components = n_components

        # define model architecture
        self.spatial_filtering = nn.Conv2d(
            1, self.n_components, (self.spatial_dim, 1), bias=False)

        self.classifier = nn.Sequential(
            nn.Linear(n_components, 2),
        )

    def forward(self, x):
        x = self.spatial_filtering(x)
        x = torch.sum(x ** 2, dim=3)
        x = torch.log(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        x = F.log_softmax(x, dim=1)
        return x

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

n_components = 30
model = CommonSpatialFilterModel(spatial_dim=epochs_data.shape[1],
                                 n_components=n_components)

# Test model works:
n_samples_test = 10
y_test = torch.randint(0, 2, (n_samples_test,))
y_pred = model.forward(torch.randn(n_samples_test, 1, *epochs_data.shape[1:]))
output = F.nll_loss(y_pred, y_test)
_, top_class = y_pred.topk(1, dim=1)


Using device: cpu


Training

In [15]:
lr = 1e-3
n_epochs = 300
patience = 100

model.to(device=device)  # move to device before creating the optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9,
                      weight_decay=1e-4)

train(model, loader_train, loader_valid, optimizer, n_epochs, patience, device)


Starting epoch 1 / 300


avg val loss: 9.3884: 100%|██████████| 1/1 [00:00<00:00, 13.33it/s]


---  Accuracy : 0.5378151535987854 


best val loss inf -> 9.3884

Starting epoch 2 / 300


avg val loss: 12.2776: 100%|██████████| 1/1 [00:00<00:00, 11.90it/s]


---  Accuracy : 0.5378151535987854 

Waiting += 1

Starting epoch 3 / 300


...

avg val loss: 18.8789: 100%|██████████| 1/1 [00:00<00:00, 13.70it/s]


---  Accuracy : 0.462184876203537 

Waiting += 1

Starting epoch 298 / 300


avg val loss: 10.0400: 100%|██████████| 1/1 [00:00<00:00, 14.71it/s]

---  Accuracy : 0.462184876203537 

Waiting += 1
Stop training at epoch 298
Best val loss : 0.7210


CommonSpatialFilterModel(
  (spatial_filtering): Conv2d(1, 30, kernel_size=(32, 1), stride=(1, 1), bias=False)
  (classifier): Sequential(
    (0): Linear(in_features=30, out_features=2, bias=True)
  )
)